In [30]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [31]:
def pull_from_csv(csv_file, row=3, header=1):
        lwp_ind = str(csv_file).lower().find('lwp')
        lwp = str(csv_file)[lwp_ind:lwp_ind+14]
        df = pd.read_csv(csv_file, header=[header], on_bad_lines='skip')
        try:
            return [lwp,*df.iloc[row].name]
        except:
            return [lwp,*df.iloc[row]]
        
        
        
        
        
def met_ratio(df, up, down, error_method='Cramer', error_label=' Error (SD)', output='both'):
    if not isinstance(up,list):
        up = [up]
    if not isinstance(down,list):
        down = [down]    
   
    up_err_name = [f'{item}{error_label}' for item in up]
    down_err_name = [f'{item}{error_label}' for item in down]
    for item in np.concatenate([up_err_name, down_err_name]).flat:
        if item not in df.columns:
            raise KeyError(f'Cannot find "{item}" in the DataFrame. Please check the given name is correct.')
    
    up_amp = df[up].astype(float).sum(axis=1)
    down_amp = df[down].astype(float).sum(axis=1)
    if error_method.lower() == 'cramer':
        up_err = (df[up_err_name].astype(float)**2).sum(axis=1) ** 0.5
        down_err = (df[down_err_name].astype(float)**2).sum(axis=1) ** 0.5
    elif error_method.lower()=='narrow':
        up_err = df[up_err_name].astype(float).max(axis=1) 
        down_err = df[down_err_name].astype(float).max(axis=1) 
    else:
        raise KeyError('Please choose "Cramer" or "Narrow" for the error_method')
    
    ratio = up_amp / down_amp
    ratio_err = ratio * ((up_err/up_amp)**2 + (down_err/down_amp)**2)**0.5 
    
    if len(up)>1:
        up_label =f'({"+".join(up)})'
    else:
        up_label = f'{"+".join(up)}'
    if len(down)>1:
        down_label = f'({"+".join(down)})'
    else:
        down_label = f'{"+".join(down)}'    
    col_name = f'{up_label}/{down_label}'   
    col_error_name = f'{col_name} Error (SD) {error_method}'
    
    if output=='both':
        return pd.DataFrame({col_name: ratio, col_error_name: ratio_err})
    elif output=='ratio':
        return pd.DataFrame({col_name: ratio})      
    elif output=='error':
        return pd.DataFrame({col_error_name: ratio_err})   
    else:
        raise KeyError('Please choose "ratio", "error" or "both" (default) as output')

In [ ]:
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY')
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/Ellie')
data_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA')

basis_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine')

sdat_files = [f for f in sorted(data_folder.rglob('*')) if ("act.sdat" in f.name.lower() and "press_35".lower() in f.name.lower())]
for file in tqdm(sdat_files):
    ref = Path(f'{str(file)[0:-8]}ref.SDAT')
    csv = Path(f'{str(file)[0:-5]}_PRESS_35_WM.csv')
    pdf = Path(f'{str(file)[0:-4]}pdf')
    print(pdf)
    if not csv.is_file():
    # if 1:    
        command = f'tarquin --rows 1 --cols 1 --input {file} --output_pdf {pdf} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}' # with pdf graph
        # command = f'tarquin --rows 1 --cols 1 --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}' # no pdf graph

        print(f'{command}\n') 
        os.system(command)
        

In [58]:
csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("press_35" in f.name.lower() and ".csv" in f.name.lower())]



rows_to_process = {
                   'WM': 0,
                   }

for label,row_num in rows_to_process.items():
    data_list = []
    for file in csv_files:
        pulled_data = [*pull_from_csv(file,row=row_num),*pull_from_csv(file,row=row_num+3),*pull_from_csv(file,header=6,row=row_num+1)]
        data_list.append(pulled_data)
        print(f'{pulled_data[0]} has {len(pulled_data)} elements')

    
        print('{')
    print(f'\n{data_list[0] = }')   
    print(f'{len(data_list[0]) = }')
    
    row0 = pull_from_csv(file,row=row_num)
    print(f'\n{row0 = }')   
    print(f'{len(row0) = }')
    
    row3 = pull_from_csv(file,row=row_num+3)
    print(f'\n{row3 = }')   
    print(f'{len(row3) = }')
    
    H6row1 = pull_from_csv(file,header=6,row=row_num+1)
    print(f'\n{H6row1 = }')   
    print(f'{len(H6row1) = }')
    
    
    
    my_columns = list(pd.read_csv(file, header=[1], on_bad_lines='skip'))
    print(f'\n{my_columns=} ')
    print(f'{len(my_columns)=} ')

    my_errors = [item+' Error (SD)' for item in my_columns]
    print(f'\n{my_errors=} ')
    print(f'{len(my_errors)=} ')


    my_diagnostics = pull_from_csv(file,row=0, header=6)    
    print(f'\n{my_diagnostics=} ')
    print(f'{len(my_diagnostics)=} ')

    total_columns = ['LWP', *my_columns, 'Error for LWP', *my_errors,*my_diagnostics ]
    print(f'\nTotal number of columns = {len(total_columns)}  ')
    df = pd.DataFrame(data_list, columns=total_columns)
    
    df.to_excel(f'{label}.xlsx', index=False)
    
    #now calculate ratios
    # df_L2N = met_ratio(df,['Lac', 'Threonine'], 'TNAA')
    # df_L2N_narrow_error = met_ratio(df,['Lac', 'Threonine'], 'TNAA', error_method='Narrow', output='error')
    # df_N2C = met_ratio(df,'TNAA', 'TCho')
    # df_C2C = met_ratio(df,'TCho', 'Cr')
    # df_N2Cr = met_ratio(df,'TNAA', 'Cr')


    # df_ratio = pd.concat((df_L2N, df_N2C, df_C2C, df_N2C, df), axis=1)
    # df_ratio = pd.concat((df_L2N,df_L2N_narrow_error, df_N2C, df_C2C, df_N2Cr, df), axis=1)

    df.set_index('LWP').to_excel(f'{label}_RATIOS_PRESS_INSPIRE.xlsx')



LWP767_evening has 105 elements
{
LWP762_morning has 105 elements
{
LWP772_morning has 105 elements
{
LWP774_evening has 105 elements
{
LWP719_morning has 105 elements
{
LWP766_evening has 105 elements
{
LWP766_evening has 105 elements
{
LWP776_evening has 105 elements
{
LWP718_morning has 105 elements
{
LWP764_evening has 105 elements
{
LWP711_morning has 105 elements
{
LWP773_morning has 105 elements
{
LWP763_morning has 105 elements
{
LWP777_evening has 105 elements
{
LWP768_morning has 105 elements
{
LWP771_morning has 105 elements
{
LWP713_morning has 105 elements
{
LWP770_morning has 105 elements
{
LWP712_morning has 105 elements
{
LWP769_evening has 105 elements
{
LWP769_morning has 105 elements
{
LWP721_morning has 105 elements
{
LWP771_evening has 105 elements
{
LWP765_morning has 105 elements
{
LWP775_morning has 105 elements
{
LWP717_morning has 105 elements
{
LWP768_evening has 105 elements
{
LWP774_morning has 105 elements
{
LWP764_morning has 105 elements
{
LWP716_morning

In [ ]:
dfbb = met_ratio(df,'TNAA', 'TCho', error_method='Narrow', output='error')
dfbb


In [24]:
my_columns

['Row',
 'Col',
 'Slice',
 '-CrCH2',
 'Ala',
 'Asp',
 'Cr',
 'GABA',
 'GPC',
 'Glc',
 'Gln',
 'Glu',
 'Gua',
 'Ins',
 'Lac',
 'Lip09',
 'Lip13a',
 'Lip13b',
 'Lip20',
 'MM09',
 'MM12',
 'MM14',
 'MM17',
 'MM20',
 'NAA',
 'NAAG',
 'PCh',
 'Scyllo',
 'Tau',
 'TNAA',
 'TCho',
 'Glx',
 'TLM09',
 'TLM13',
 'TLM20']

In [25]:
my_columns

['Row',
 'Col',
 'Slice',
 '-CrCH2',
 'Ala',
 'Asp',
 'Cr',
 'GABA',
 'GPC',
 'Glc',
 'Gln',
 'Glu',
 'Gua',
 'Ins',
 'Lac',
 'Lip09',
 'Lip13a',
 'Lip13b',
 'Lip20',
 'MM09',
 'MM12',
 'MM14',
 'MM17',
 'MM20',
 'NAA',
 'NAAG',
 'PCh',
 'Scyllo',
 'Tau',
 'TNAA',
 'TCho',
 'Glx',
 'TLM09',
 'TLM13',
 'TLM20']

In [47]:
33+35+35+2

105

In [48]:
36+36+33

105